# 数值运算方法

`NumPy` 的基本能力之一是快速对每个元素进行运算，既包括基本算术运算（加、减、乘、除），也包括更复杂的运算（三角函数、指数函数和对数函数等）。  
`Pandas` 继承了 `NumPy` 的功能，在 2.3 节介绍过的通用函数是关键。但是 `Pandas` 也实现了一些高效技巧：  
对于一元运算（像函数与三角函数），这些通用函数将在输出结果中保留索引和列标签；  
而对于二元运算（如加法和乘法），`Pandas` 在传递通用函数时会自动对齐索引进行计算。  
这就意味着，保存数据内容与组合不同来源的数据——两处在 `NumPy` 数组中都容易出错的地方——变成了 `Pandas` 的杀手锏。

In [1]:
import numpy as np
import pandas as pd

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
  img:nth-child(1) {width:30%; height: 30%;}
</style>

## 1. 通用函数：保留索引

因为 `Pandas` 是建立在 `NumPy` 基础之上的，所以 `NumPy` 的通用函数同样适用于 `Pandas` 的 `Series` 和 `DataFrame` 对象。

In [3]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [4]:
df = pd.DataFrame(rng.randint(0, 10, (3,4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [5]:
np.exp(ser)  # 如果对这两个对象的其中一个使用 NumPy 通用函数，生成的结果是另一个保留索引的 Pandas 对象

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [6]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## 2. 通用函数：索引对齐

当在两个 `Series` 或 `DataFrame` 对象上进行二元计算时，`Pandas` 会在计算过程中对齐两个对象的索引。当你处理不完整的数据时，这一点非常方便。

### 2.1. Series索引对齐

In [7]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662, 'California': 423967}, name='area')                     # 面积最大三个州的面积
population = pd.Series({'California': 38332521, 'Texas': 26448193, 'New York': 19651127}, name='population')  # 人口最多三个州的人口

In [8]:
population / area  # 结果数组的索引是两个输入数组索引的并集

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [9]:
# area.index | population.index
area.index.union(population.index)

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

In [10]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [11]:
A.add(B, fill_value=0)  # 如果用 NaN 值不是我们想要的结果，那么可以用适当的对象方法代替运算符，也可以设置参数自定义 A 或 B 缺失的数据。

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### 2.2. DataFrame索引对齐

在计算两个 `DataFrame` 时，类似的索引对齐规则也同样会出现在共同（并集）列中。

In [12]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [13]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [14]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


In [15]:
fill = A.stack().mean()  # 用 A 中所有值的均值来填充缺失值（计算 A 的均值需要用 stack 将二维数组压缩成一维数组）
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


**Python运算符与Pandas方法的映射关系**

| Python运算符 | Pandas方法 |
| -- | -- |
| + | add() |
| - | sub()、subtract() |
| * | mul()、multiply() |
| / | truediv()、div()、divide() |
| // | floordiv() |
| % | mod() |
| ** | pow() |

## 3. 通用函数：DataFrame与Series的运算

`DataFrame` 和 `Series` 的运算规则，与 `NumPy` 中二维数组与一维数组的运算规则是一样的。

In [16]:
A = rng.randint(10, size=(3,4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [17]:
A - A[0]  # 根据 NumPy 的广播规则，让二维数组减自身的一行数据会按行计算

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

In [18]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


In [19]:
df.subtract(df['R'], axis=0)  # 如果你想按列计算，那么就需要利用前面介绍过的运算符方法，通过 axis 参数设置

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


In [20]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int64

In [21]:
df - halfrow  # DataFrame / Series 的运算与前面介绍的运算一样，结果的索引都会自动对齐

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


这些行列索引的保留与对齐方法说明 `Pandas` 在运算时会一直保存这些数据内容，从而避免在处理数据类型有差异和 \/ 或维度不一致的 `NumPy` 数组时可能遇到的问题。